# NLP Final Project - Wikipedia Search Engine

## 4 - Research

We load all the data that we computed before

In [2]:
import json
# this two functions are the same as in the cleaning part so we apply exactly the same preprocess to the query as to the paragraphs
from utils import clean_text, get_n_gram
import re

n_grams = {}

with open('../data/1_grams.json') as f:
    n_grams['1'] = json.load(f)

with open('../data/2_grams.json') as f:
    n_grams['2'] = json.load(f)

with open('../data/3_grams.json') as f:
    n_grams['3'] = json.load(f)

with open('../data/4_grams.json') as f:
    n_grams['4'] = json.load(f)

with open('../data/raw_wiki_with_ids.json') as f:
    wiki = json.load(f)

MemoryError: 

We need a function that can find a paragraph in the raw database from a given id

In [69]:
def find_paragraph(id_paragraph):
    """
    This function returns the paragraph of the wiki that has the id given as parameter

    Args:
    id_paragraph: int, id of the paragraph to be found
    """
    for page in wiki:
        for section in page['content']:
            if id_paragraph in section['ids']:
                text = section['paragraphs'][section['ids'].index(id_paragraph)]
                # formatted section title
                section_title = section['title'].replace(' ', '_')
                # remove references in square brackets
                section_title = re.sub(r'\[.*?\]', '', section_title)
                url = page['url'] + '#' + section_title
                return text, url
        if 'summary_ids' in page:
            if id_paragraph in page['summary_ids']:
                text = page['summary'][page['summary_ids'].index(id_paragraph)]
                url = page['url']
                return text, url
            
find_paragraph(0)

('Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation.[31]\n',
 'https://en.wikipedia.org/wiki/Python_(programming_language)')

In [68]:
def count_ngram_frequency(query_ngrams, db_ngrams):
    """
    This function returns the frequency of the n-grams in the query in the paragraphs

    Args:
    query_ngrams: list of str, n-grams of the query
    db_ngrams: dict, n-grams of the paragraphs
    """
    n_grams_frequency = {n_gram: {} for n_gram in query_ngrams}
    for word in query_ngrams:
        # if the word is not in the db_ngrams we skip it
        if word in db_ngrams:
            word_par = db_ngrams[word]
            # we iterate over the paragraphs where the word is present
            for par in word_par:
                # if the paragraph is already in the n_grams_frequency we increment the frequency
                if par in n_grams_frequency[word]:
                    n_grams_frequency[word][par] += 1
                # if not we add the paragraph to the dictionary
                else:
                    n_grams_frequency[word][par] = 1
    # we return the dictionary with the frequency of the n-grams in the documents sorted by frequency
    return {n_gram: {k: v for k, v in sorted(par.items(), key=lambda item: item[1], reverse=True)} for n_gram, par in n_grams_frequency.items()}

We will try to find how many times a paragraph has the n-grams in its text. For this part, we do not care about how many times the n-gram appears in the paragraph, we just want to know if it appears at least once.

In [67]:
def get_unique_ngram(ngram_freq):
    """
    This function counts the number of unique n-grams coming from the query in the paragraphs

    Args:
    ngram_freq: dict, frequency of the n-grams in the paragraphs
    """
    par_unique_ngram = {}
    # count how many unique n_grams are in each paragraph
    for _, paragraphs in ngram_freq.items():
        for par in paragraphs:
            added = False
            # if the paragraph is already in the dictionary we increment the count
            if par in par_unique_ngram:
                # we increment the count of the paragraph only if the n-gram has not already been detected in the paragraph
                if not added:
                    par_unique_ngram[par] += 1
                    added = True
            else:
                par_unique_ngram[par] = 1

    # sort the paragraphs by the number of unique n_grams
    return {k: v for k, v in sorted(par_unique_ngram.items(), key=lambda item: item[1], reverse=True)}

We will compute a score for each paragraph and for each n of n-grams. The score will be the number of times the n-gram appears in the paragraph.

In [70]:
def get_scores(ngrams_query, ngrams_db, n):
    """
    This function returns the scores of the paragraphs based on the each n-grams

    Args:
    ngrams_query: dict, n-grams of the query
    ngrams_db: dict, n-grams of the paragraphs
    n: int, max n-gram to be considered
    """
    scores = {str(n): {} for n in range(1, n+1)}
    n_gram_frequencies = {str(n): {} for n in range(1, n+1)}
    for n_gram in ngrams_query:
        n_gram_frequencies[n_gram] = count_ngram_frequency(ngrams_query[n_gram], ngrams_db[n_gram])
        scores[n_gram] = get_unique_ngram(n_gram_frequencies[n_gram])

    return scores, n_gram_frequencies

We then combine the scores of each n-gram to get a final score for the paragraph. We multiply the score of each n-gram by a weight that depends on the length of the n-gram. 

The bigger n is, the more important the n-gram is as it means that the looked words are closer to each other.

In [71]:
def combine_scores(scores, n_gram_frequencies, weights):
    """
    This function returns the combined scores of the paragraphs based on the scores of the n-grams

    Args:
    scores: dict, scores of the paragraphs based on the n-grams
    n_gram_frequencies: dict, frequency of the n-grams in the paragraphs
    weights: dict, weights of the n-grams
    """
    # the score of the paragraph is the sum of the scores of the n_grams multiplied by the weight of the n_gram
    combined_scores = {}
    for n_gram in scores:
        for par, score in scores[n_gram].items():
            if par in combined_scores:
                combined_scores[par] += score * weights[n_gram] + n_gram_frequencies[n_gram][par] if par in n_gram_frequencies[n_gram] else score * weights[n_gram]
            else:
                combined_scores[par] = score * weights[n_gram] + n_gram_frequencies[n_gram][par] if par in n_gram_frequencies[n_gram] else score * weights[n_gram]

    return {k: v for k, v in sorted(combined_scores.items(), key=lambda item: item[1], reverse=True)}

In [77]:
# this n represents the maximum n-gram we want to use
n = 4
max_returned_paragraphs = 5
# these are the weights of the n-grams.
weights = {
    '1': 1,
    '2': 10,
    '3': 100,
    '4': 1000
}
query = 'When was the first computer invented?'

In [78]:
def search(query, n_grams, n, weights):
    """
    This function returns the paragraphs ids that are the most relevant to the query and their scores

    Args:
    query: str, query to be searched
    n_grams: dict, n-grams of the paragraphs
    n: int, max n-gram to be considered
    """
    cleaned_query = clean_text(query)
    ngrams_query = {str(n): get_n_gram(n, cleaned_query) for n in range(1, n+1)}
    scores, n_gram_frequencies = get_scores(ngrams_query, n_grams, n)
    ranking = combine_scores(scores, n_gram_frequencies, weights)
    return ranking

In [1]:
with open('../data/sorted_links.json') as f:
    links = json.load(f)
def link_score(link):
    """
    This function returns the score of the link

    Args:
    link: str, link to be scored
    """
    # the score is the number of citations of the link divided by 10
    return links[link] / 10

link_score('https://en.wikipedia.org/wiki/Computer')

NameError: name 'json' is not defined

In [79]:
ranking = search(query, n_grams, n, weights)
# we get the top paragraphs
ranking = list(ranking.keys())[:max_returned_paragraphs]
for paragraph in ranking:
    text, url = find_paragraph(paragraph)
    print(text + '\n' + url)
    print('---------------------------------')


Wilhelm Schickard designed and constructed the first working mechanical calculator in 1623.[18] In 1673, Gottfried Leibniz demonstrated a digital mechanical calculator, called the Stepped Reckoner.[19] Leibniz may be considered the first computer scientist and information theorist, because of various reasons, including the fact that he documented the binary number system. In 1820, Thomas de Colmar launched the mechanical calculator industry[note 1] when he invented his simplified arithmometer, the first calculating machine strong enough and reliable enough to be used daily in an office environment. Charles Babbage started the design of the first automatic mechanical calculator, his Difference Engine, in 1822, which eventually gave him the idea of the first programmable mechanical calculator, his Analytical Engine.[20] He started developing this machine in 1834, and "in less than two years, he had sketched out many of the salient features of the modern computer".[21] "A crucial step was

We remarked at the end of the project that we have duplicates in the database.

The reason of these duplicates is that in the scrapping phase, we took care to exclude the links of the form https://en.wikipedia.org/wiki/Title once they have been visited. However, we did not take care of the links of the form https://en.wikipedia.org/wiki/Title#section that are considered as different links by the scrapping algorithm.

As the scrapping phase is very time consuming, we decided to keep the duplicates and to not recompute the scrapping phase. Moreover, the duplicates must not be very frequent.

In [81]:
!jupyter nbconvert --to html search.ipynb

[NbConvertApp] Converting notebook search.ipynb to html
[NbConvertApp] Writing 314368 bytes to search.html
